In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date

from scipy.stats import zscore
# from scipy.io import wavfile

from sklearn.model_selection import KFold
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score,\
                            confusion_matrix, \
                            precision_score, \
                            recall_score, \
                            f1_score, \
                            classification_report


from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, \
                            r2_score

from prettytable import PrettyTable

In [2]:
data_path = ['./NYC_Airbnb/development.csv', './NYC_Airbnb/evaluation.csv']

def load_data(data_path, phase):

    if phase == 'dev':
        data_path = data_path[0]
        dataset = pd.read_csv(data_path, low_memory=False)

    elif phase == 'eval':
        data_path = data_path[1]
        dataset = pd.read_csv(data_path, low_memory=False)

    else:
        print('Phase not valid')
        raise Exception

    return dataset

In [3]:
df = load_data(data_path=data_path, phase='dev')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39116 entries, 0 to 39115
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39116 non-null  int64  
 1   name                            39103 non-null  object 
 2   host_id                         39116 non-null  int64  
 3   host_name                       39097 non-null  object 
 4   neighbourhood_group             39116 non-null  object 
 5   neighbourhood                   39116 non-null  object 
 6   latitude                        39116 non-null  float64
 7   longitude                       39116 non-null  float64
 8   room_type                       39116 non-null  object 
 9   price                           39116 non-null  int64  
 10  minimum_nights                  39116 non-null  int64  
 11  number_of_reviews               39116 non-null  int64  
 12  last_review                     

In [33]:
(df[df["number_of_reviews"]==0].index == df[df["last_review"].isna()].index).all(),\
(df[df["number_of_reviews"]==0].index == df[df["reviews_per_month"].isna()].index).all()

(True, True)

In [34]:

df["reviews_per_month"].fillna(0, inplace=True) # with inplace, we make the changes directly to df

In [35]:
# Now, for the next part, we need to convert any non-numerical (i.e. categorical) feature into a
# numerical one. This is because machine learning models work on numeric data and cannot digest
# non-numeric data without some kind of transformation.
# From our previous assessment we have identified three categorical features that need to be transformed
# into numerical ones: neighbourhood_group, neighbourhood, room_type.

df_1h = pd.get_dummies(df, columns=['neighbourhood_group', 'neighbourhood', 'room_type'])

In [36]:
df.shape, df_1h.shape


((39116, 16), (39116, 242))

In [37]:
# drop unused columns
df_dropped = df_1h.drop(columns=["host_id", "name", "host_name", "last_review"])

# define the mask for the training/validation samples (those with a price, the others
# will belong to the test set)
train_valid_mask = ~df_dropped["price"].isna()
# extract the feature names (for later use)
feature_names = df_dropped[train_valid_mask].drop(columns=["price"]).columns

X = df_dropped.drop(columns=["price"]).values
y = df_dropped["price"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]

# X_test = X[~train_valid_mask]
# y_test = y[~train_valid_mask]

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

In [38]:
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train , y_train)

r2_score(y_valid, reg.predict(X_valid))

0.07970260075670399

In [39]:
sorted(zip(feature_names, reg.feature_importances_), key=lambda x: x[1], reverse=True)

[('longitude', 0.16722396394712963),
 ('latitude', 0.15237493276830008),
 ('id', 0.12773199310440395),
 ('availability_365', 0.0828964022053185),
 ('minimum_nights', 0.07481450509500819),
 ('room_type_Entire home/apt', 0.06307473812272037),
 ('reviews_per_month', 0.056779475371180124),
 ('number_of_reviews', 0.042375717277390784),
 ('calculated_host_listings_count', 0.040358813643678076),
 ('neighbourhood_Astoria', 0.0388253160973694),
 ('neighbourhood_Battery Park City', 0.02003548983077795),
 ('neighbourhood_Upper West Side', 0.012632956723725651),
 ('neighbourhood_Lower East Side', 0.01260413646717635),
 ('neighbourhood_Bedford-Stuyvesant', 0.010109874159663483),
 ('neighbourhood_Clinton Hill', 0.008159296530672946),
 ('neighbourhood_Randall Manor', 0.006261593201417485),
 ('neighbourhood_Midtown', 0.004723979930257032),
 ('neighbourhood_Tribeca', 0.0043147048958018),
 ('neighbourhood_Chelsea', 0.004101135814081659),
 ('neighbourhood_Williamsburg', 0.004078839790463962),
 ('neighbou

Interestingly, the model assigns a high feature importance to the latitudes and longitudes, and a
much lower importance to the neighbourhood information. This means that the model extracts
more meaningful information from the “raw” data (as previously discussed), and makes little to
no use of the neighbourhood information.
Considering that the majority of the columns in our dataset has been generated from neighbourhood
information (by 1-hot encoding the original columns), it is reasonable to discard those
columns altogether. By doing that, the random forest will not have as many “noisy” features to
select from at each split. As a consequence it will be more likely that, at each split, more useful
features will be available, thus building better trees.
We can redo the entire preprocessing step as follows:

In [40]:
# Only encode "room_type"
df_1h = pd.get_dummies(df, columns=['room_type'])

# discard "neighbourhood" and "neighbourhood_group"
df_dropped = df_1h.drop(columns=["neighbourhood_group","neighbourhood","host_id", "name", "host_name", "last_review"])

train_valid_mask = ~df_dropped["price"].isna()

feature_names = df_dropped[train_valid_mask].drop(columns=["price"]).columns

X = df_dropped.drop(columns=["price"]).values
y = df_dropped["price"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]

# X_test = X[~train_valid_mask]
# y_test = y[~train_valid_mask]

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train, y_train)

r2_score(y_valid, reg.predict(X_valid))

0.1162855504260838

In [41]:

sorted(zip(feature_names, reg.feature_importances_), key=lambda x: x[1],reverse=True)

[('longitude', 0.23848913987918202),
 ('id', 0.1944035055823323),
 ('latitude', 0.18207323673033954),
 ('availability_365', 0.08610408843715538),
 ('minimum_nights', 0.08542812228591133),
 ('room_type_Entire home/apt', 0.06307473812272034),
 ('reviews_per_month', 0.06197904085057712),
 ('calculated_host_listings_count', 0.05088126055940794),
 ('number_of_reviews', 0.03690226048213828),
 ('room_type_Private room', 0.00037693522989826103),
 ('room_type_Shared room', 0.00028767184033743796)]

We can now see that the model is giving even more importance to the latitude and longitude. This
is because we have removed any other source of location information.
Now, let us try to re-introduce two of the features we previously discarded: id and host_id. We
will first introduce the first one, then the second one, then both together. For each configuration
of features we will train a separate model and assess how it behaves. If our initial hypotheses are
correct, these two features should not have a particular impact on our model’s performance

In [42]:

for include_features in [["id"], ["host_id"], ["id", "host_id"]]:
    df_1h = pd.get_dummies(df, columns=['room_type'])

    # Extract the "id" information
    if "id" in include_features:
        df_1h["id"] = df_1h.index
    df_dropped = df_1h.drop(columns=["neighbourhood_group","neighbourhood", "name", "host_name", "last_review"])

    # if "host_id" should not be kept, it is discarded
    if "host_id" not in include_features:
        df_dropped = df_dropped.drop(columns=["host_id"])

    train_valid_mask = ~df_dropped["price"].isna()

    feature_names = df_dropped[train_valid_mask].drop(columns=["price"]).columns

    X = df_dropped.drop(columns=["price"]).values
    y = df_dropped["price"].values

    X_train_valid = X[train_valid_mask]
    y_train_valid = y[train_valid_mask]

    # X_test = X[~train_valid_mask]
    # y_test = y[~train_valid_mask]

    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

    reg = RandomForestRegressor(100, random_state=42)
    reg.fit(X_train, y_train)

    print(include_features, r2_score(y_valid, reg.predict(X_valid)))

['id'] 0.08385550018096222
['host_id'] 0.13692655210347704
['id', 'host_id'] 0.1043008455575859


In [43]:
sorted(zip(feature_names, reg.feature_importances_), key=lambda x: x[1],
reverse=True)

[('longitude', 0.21488045414599827),
 ('host_id', 0.19339035781321365),
 ('latitude', 0.1383893491388881),
 ('id', 0.10739261621014184),
 ('minimum_nights', 0.07956772018890616),
 ('availability_365', 0.07705175351026355),
 ('room_type_Entire home/apt', 0.06307473812272034),
 ('calculated_host_listings_count', 0.04477211867690946),
 ('reviews_per_month', 0.04431090224494947),
 ('number_of_reviews', 0.03614330795840609),
 ('room_type_Shared room', 0.0005238879454123916),
 ('room_type_Private room', 0.0005027940441908334)]

In [44]:
months = (df_1h["number_of_reviews"] / df_1h["reviews_per_month"])
months = months.fillna(months.mean())

In [45]:

np.corrcoef([months, df.index])[0][1], np.corrcoef([months, df["host_id"]])[0][1]

(0.0015422307379890398, -0.4443239608192033)

In [47]:
# We also know that many of the entries in our dataset have received no reviews. For all these
# entries, it will be impossible for us to work out the post’s age. More specifically, we can compute
# the fraction of posts with no reviews as follows:

len( df_1h[df_1h["number_of_reviews"] == 0])/len(df_1h)

0.20556805399325084

Approximately 20% of the entries in our dataset cannot be assigned a “timestamp”, if not through
their id. On the other hand, we do have the timestamp information, in terms of id, for all entries.
For this reason, we will be keeping both id and host_id in our dataset. Do keep in mind, though,
that in many (most) cases, ids will not be informative in the least (especially when you also have
reliable information on creation dates, or when ids are generated randomly).
Finally, we can consider exploring the title of posts (name attribute). This is a natural language
string which may contain useful information we have been sitting on this whole time.
We can process the textual data using sklearn’s TfidfVectorizer, which will split each title into
tokens and remove stopwords. We can consider using a binary feature for each of the most popular
words, since we will only consider the presence/absence of terms as relevant (binary term
frequency, with no inverse document frequency).


In [49]:
vectorizer = TfidfVectorizer(stop_words="english", binary=True, use_idf=False, norm=False)
# word presence matrix (i-th row, j-th col => 1 if j-th word is contained in i-th title)
wpm = vectorizer.fit_transform(df_1h["name"].fillna(""))

In [50]:
N = 150
freq = sorted(zip(vectorizer.get_feature_names(), wpm.sum(axis=0).tolist()[0]), key=lambda x: x[1], reverse=True)[:N]

In [51]:
freq


[('room', 8146.0),
 ('bedroom', 6483.0),
 ('private', 5826.0),
 ('apartment', 5399.0),
 ('cozy', 4077.0),
 ('apt', 3836.0),
 ('brooklyn', 3341.0),
 ('studio', 3286.0),
 ('spacious', 3054.0),
 ('manhattan', 2882.0),
 ('park', 2498.0),
 ('east', 2448.0),
 ('sunny', 2346.0),
 ('williamsburg', 2188.0),
 ('beautiful', 2011.0),
 ('near', 1923.0),
 ('nyc', 1882.0),
 ('village', 1875.0),
 ('heart', 1686.0),
 ('loft', 1681.0),
 ('large', 1659.0),
 ('bed', 1623.0),
 ('modern', 1460.0),
 ('central', 1453.0),
 ('luxury', 1366.0),
 ('bright', 1361.0),
 ('home', 1291.0),
 ('new', 1284.0),
 ('1br', 1282.0),
 ('west', 1272.0),
 ('location', 1253.0),
 ('bushwick', 1168.0),
 ('charming', 1082.0),
 ('upper', 1068.0),
 ('br', 1020.0),
 ('midtown', 1010.0),
 ('quiet', 984.0),
 ('brownstone', 973.0),
 ('clean', 933.0),
 ('great', 923.0),
 ('harlem', 914.0),
 ('square', 894.0),
 ('close', 833.0),
 ('subway', 825.0),
 ('garden', 811.0),
 ('bath', 808.0),
 ('huge', 808.0),
 ('heights', 760.0),
 ('times', 727.0

As you can see, the most frequent words can also be quite useful for extracting additional information
such as spaces available (e.g. balcony, backyard), place conditions (e.g. renovated, modern,
sunny) and even information on the possible price ranges (e.g. luxury).
Our wpm matrix already contains binary values based on whether words are present or not. We
can use this matrix, with only the N most frequent columns, to build an additional DataFrame to
be attached to the original one.

In [52]:
# mask to be used to filter columns in wpm (only keeps the ones for the 100 most frequent words)

words = [ word for word, _ in freq ]

mask = [ w in words for w in vectorizer.get_feature_names() ]

words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
words_df = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(), columns=[f"word_{word}" for word in words_], index=df_1h.index)

In [53]:
# Only encode "room_type"
df_1h = pd.get_dummies(df, columns=['room_type'])
df_1h = df_1h.join(pd.DataFrame(data=wpm[:, np.array(mask)].toarray(), columns=[f"word_{word}" for word in words_], index=df_1h.index))

# discard "neighbourhood" and "neighbourhood_group"
df_dropped = df_1h.drop(columns=["neighbourhood_group","neighbourhood", "name", "host_name", "last_review"])
df_dropped["id"] = df_dropped.index

train_valid_mask = ~df_dropped["price"].isna()

feature_names = df_dropped[train_valid_mask].drop(columns=["price"]).columns

X = df_dropped.drop(columns=["price"]).values
y = df_dropped["price"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]

X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train, y_train)

r2_score(y_valid, reg.predict(X_valid))

0.1503997321706172

In [ ]:

param_grid = {
"n_estimators": [100, 250, 500],
"criterion": ["mse", "mae"],
"max_features": ["auto", "sqrt", "log2"],
"random_state": [42], # always use the samet random seed
"n_jobs": [-1], # for parallelization
}
gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring="r2", n_jobs=-1, cv=5)

gs.fit(X_train_valid, y_train_valid)
gs.best_score_